In [1515]:
import numpy as np
import pandas as pd
import pandas_bokeh;

In [1516]:
# Embedding plots in Jupyter/Colab Notebook
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [1517]:
# For exporting plots as HTML
pandas_bokeh.output_file(filename='birds_dashboard.html',title='Birds Dashboard')

In [1518]:
# Get data from google sheets
sheet_id = '1dENiPTJtdqOiU9Oiwxen2UU2K4HP1jOR_MyGcjQP81k'
sheet_name = 'registro'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df = pd.read_csv(url, usecols=[0,2,3,4])

In [1519]:
# Get datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Fecha        187 non-null    object 
 1   Descripción  187 non-null    object 
 2   Cantidad(g)  187 non-null    float64
 3   Costo($)     399 non-null    float64
dtypes: float64(2), object(2)
memory usage: 12.6+ KB


In [1520]:
# Change datatypes
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Month'] = df['Fecha'].dt.month

In [1521]:
# Data for >>Pie Plot<< and >>Scatter Plot<<
df.head()

,Fecha,Descripción,Cantidad(g),Costo($),Month
0,2021-03-01,Papaya,227.0,385.9,3.0
1,2021-03-01,Plátano,165.0,379.5,3.0
2,2021-03-02,Plátano,164.0,377.2,3.0
3,2021-03-03,Banano,148.0,355.2,3.0
4,2021-03-03,Banano,124.0,297.6,3.0


In [1522]:
# Group by month -> Data for >>Simple Line Plot<< and >>Bar Plot<<

by_month = df.groupby(['Month']).sum()
by_month = by_month.rename(index={
    1.0:'Jan', 2.0:'Feb', 3.0:'Mar', 4.0:'Apr', 5.0:'May', 6.0:'Jun',
    7.0:'Jun', 8.0:'Ago', 9.0:'Sep', 10.0:'Oct', 11.0:'Nov', 12.0:'Dic'})
by_month.head()

,Cantidad(g),Costo($)
Month,,
Mar,4500.0,10207.9
Apr,3195.0,7317.9
May,6639.0,16558.1
Jun,5680.0,13177.6
Jun,7780.0,17926.7


In [1523]:
# Group by description and month
by_fruit = df.groupby(['Month','Descripción']).sum()
by_fruit.head()

Cantidad(g)  Costo($)
Month Descripción                       
3.0   Banano            1273.0    3055.2
      Papaya             449.0     763.3
      Plátano           2778.0    6389.4
4.0   Banano             396.0     950.4
      Papaya             117.0     198.9

In [1524]:
# Subset by cant -> Data for >>Stacked Bar Plot<<
by_fruit = by_fruit.reset_index()
df_cant = by_fruit.drop(['Costo($)'], axis=1)
df_cant.head()


,Month,Descripción,Cantidad(g)
0,3.0,Banano,1273.0
1,3.0,Papaya,449.0
2,3.0,Plátano,2778.0
3,4.0,Banano,396.0
4,4.0,Papaya,117.0


In [1525]:
# Pivot subset cant
df_cant = df_cant.pivot(columns='Descripción', index='Month', values='Cantidad(g)')
df_cant = df_cant.rename(index={
    1.0:'Jan', 2.0:'Feb', 3.0:'Mar', 4.0:'Apr', 5.0:'May', 6.0:'Jun',
    7.0:'Jun', 8.0:'Ago', 9.0:'Sep', 10.0:'Oct', 11.0:'Nov', 12.0:'Dic'})
df_cant= df_cant.groupby('Month', sort=False).sum()
df_cant.head()

Descripción,Banano,Naranja,Papaya,Plátano
Month,,,,
Mar,1273.0,0.0,449.0,2778.0
Apr,396.0,0.0,117.0,2682.0
May,799.0,1097.0,1094.0,3649.0
Jun,1463.0,0.0,0.0,11997.0
Ago,0.0,0.0,0.0,5563.0


In [1526]:
#Subset by cost -> Data fo >>Multiline Plot<<
df_cost = by_fruit.drop(['Cantidad(g)'], axis=1)
df_cost.head()

,Month,Descripción,Costo($)
0,3.0,Banano,3055.2
1,3.0,Papaya,763.3
2,3.0,Plátano,6389.4
3,4.0,Banano,950.4
4,4.0,Papaya,198.9


In [1527]:
# Pivot subset cost
df_cost = df_cost.pivot(columns='Descripción', index='Month', values='Costo($)')
df_cost = df_cost.rename(index={
    1.0:'Jan', 2.0:'Feb', 3.0:'Mar', 4.0:'Apr', 5.0:'May', 6.0:'Jun',
    7.0:'Jun', 8.0:'Ago', 9.0:'Sep', 10.0:'Oct', 11.0:'Nov', 12.0:'Dic'})
df_cost = df_cost.groupby('Month', sort=False).sum()
df_cost.head()

Descripción,Banano,Naranja,Papaya,Plátano
Month,,,,
Mar,3055.2,0.0,763.3,6389.4
Apr,950.4,0.0,198.9,6168.6
May,1917.6,4388.0,1859.8,8392.7
Jun,3511.2,0.0,0.0,27593.1
Ago,0.0,0.0,0.0,12794.9


# Plotting

In [1528]:
# Plot 1. Simple Line Plot
line_plot= by_month.plot_bokeh(kind="line", y="Cantidad(g)", color='#6C3FE3', plot_data_points=True, legend=False , xlabel='' ,show_figure=False)

In [1529]:
# Plot 2. Pie Plot
pie_plot= df.groupby(['Descripción']).sum().plot_bokeh.pie(y='Cantidad(g)',colormap='Colorblind', title= 'Fruit eaten by birds in my window in 2021',show_figure=False)

__x__values_original


In [1530]:
# Plot 3. Stacked Bar Plot
stackedbar_plot = df_cant.plot_bokeh(kind='bar', stacked=True, colormap='Colorblind' ,xlabel='', show_figure=False)

In [1531]:
# Plot 4. Multiline Plot
multiline_plot = df_cost.plot_bokeh(kind='line', colormap='Colorblind', ylabel='Costo($)', show_figure=False)

In [1532]:
# Plot 5. Bar Plot
bar_plot = by_month.plot_bokeh(kind="bar", y='Costo($)', ylabel='', color='#8561E5', legend=False, show_figure=False)

In [1533]:
# Plot 6. Scatter Plot
scatter_plot = df.plot_bokeh(kind="scatter", x='Cantidad(g)', y='Costo($)', ylabel='',category='Descripción', legend = "bottom_right", colormap='Bokeh',show_figure=False)

In [1534]:
#Make Dashboard with Grid Layout: 
pandas_bokeh.plot_grid([[line_plot, pie_plot, stackedbar_plot], 
    [multiline_plot, bar_plot, scatter_plot]], 
    width=600, toolbar_location="right")

Row(id='70432', ...)